In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    # rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, dones, rates

In [4]:
def actor(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('actor', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def generator(actions, state_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=actions, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=state_size)        
        return logits

In [6]:
def discriminator(states, actions, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=1)        
        return logits

In [8]:
def model_loss(state_size, action_size, hidden_size,
               states, actions, next_states, dones, rates):
    actions_logits = actor(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ###############################################
    next_states_logits = generator(actions=actions_logits, hidden_size=hidden_size, state_size=state_size)
    next_states_labels = tf.nn.sigmoid(next_states)
    aloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits, 
                                                                    labels=next_states_labels))
    ####################################################
    dQs = discriminator(actions=actions_labels, hidden_size=hidden_size, states=states, action_size=action_size)
    rates = tf.reshape(rates, shape=[-1, 1])
    dloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=dQs, # GAN
                                                                   labels=rates)) # 0-1
    ####################################################
    gQs = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states, action_size=action_size, 
                        reuse=True)
    dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.zeros_like(gQs))) # 0-1
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.ones_like(gQs))) # 0-1
    #####################################################
    next_actions_logits = actor(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    gQs2 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states, 
                         action_size=action_size, reuse=True)
    gQs2 = tf.reshape(gQs2, shape=[-1]) * (1-dones)
    # dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
    #                                                                 labels=tf.zeros_like(gQs2))) # 0-1
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
                                                                     labels=tf.ones_like(gQs2))) # 0-1
    # ##################################################### repeatable!
    # next_states_logits = generator(actions=next_actions_logits, hidden_size=hidden_size, state_size=state_size, 
    #                                reuse=True)
    # next_actions_logits = actor(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
    #                             reuse=True)
    # gQs3 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states_logits, 
    #                      action_size=action_size, reuse=True)
    # dones2 = tf.concat(axis=0, values=[dones[1:], tf.ones(shape=[1])])
    # gQs3 = tf.reshape(gQs3, shape=[-1]) * (1-dones2)
    # dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
    #                                                                 labels=tf.zeros_like(gQs3))) # 0-1
    # aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
    #                                                                  labels=tf.ones_like(gQs3))) # 0-1
    return actions_logits, aloss, dloss, aloss2

In [9]:
def model_opt(a_loss, d_loss, a_loss2, a_learning_rate, d_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('actor')]
    #g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        d_opt = tf.train.AdamOptimizer(d_learning_rate).minimize(d_loss, var_list=d_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, d_opt, a_opt2

In [10]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, d_learning_rate):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.dones, self.rates = model_input(state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.d_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.d_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        d_loss=self.d_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        d_learning_rate=d_learning_rate)

In [11]:
# def sample(buffer, batch_size):
#     idx = np.random.choice(np.arange(len(buffer)), size=batch_size, replace=False)
#     return [buffer[ii] for ii in idx]

In [12]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [13]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [14]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
d_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size: 200/500 a successfull episode size
# gamma = 0.99                   # future reward discount

In [15]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size,
              a_learning_rate=a_learning_rate, 
              d_learning_rate=d_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [16]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        state = env.reset()
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
# aloss_list, dloss_list, aloss2_list = [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0 # each episode
        aloss_batch, dloss_batch, aloss2_batch = [], [], []
        state = env.reset() # each episode
        num_step = 0 # each episode

        # Training steps/batches
        while True:
            # Explore (Env) or Exploit (Model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            rate = -1
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Rating the memory
            if done is True:
                rate = total_reward/500 # update rate at the end/ when episode is done
                for idx in range(num_step): # episode length
                    if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                        memory.buffer[-1-idx][-1] = rate # rate the trajectory/data
                        
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                #rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                #rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                #if np.count_nonzero(dones)==1 and len(dones) >= 1 and np.max(rates) > 0:
                if len(dones) > 1:
                    # print('np.count_nonzero(dones)==1 and len(dones) >= 1 and np.max(rates) > 0: ', 
                    #       np.count_nonzero(dones), len(dones), np.max(rates))
                    break
            #             if np.count_nonzero(dones)!=1 and len(dones) < 1 and np.max(rates) <= 0:
            #                 print(np.count_nonzero(dones), len(dones), np.max(rates))
            #                 break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                feed_dict = {model.states: states, 
                                            model.actions: actions,
                                            model.next_states: next_states,
                                            #model.rewards: rewards,
                                            model.dones: dones,
                                            model.rates: rates})
            dloss, _ = sess.run([model.d_loss, model.d_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               #model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              #model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print('dones:', 
            #       len(dones), np.count_nonzero(dones), 
            #       len(dones1), np.count_nonzero(dones1), 
            #       len(dones2), np.count_nonzero(dones2))
            aloss_batch.append(aloss)
            dloss_batch.append(dloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'dloss:{:.4f}'.format(np.mean(dloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        # gloss_list.append([ep, np.mean(gloss_batch)])
        # dloss_list.append([ep, np.mean(dloss_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:17.0000 R:17.0000 rate:0.0340 aloss:1.3947 dloss:1.5957 aloss2:1.3126 exploreP:0.9983
Episode:1 meanR:26.5000 R:36.0000 rate:0.0720 aloss:1.3978 dloss:1.5955 aloss2:1.3154 exploreP:0.9948
Episode:2 meanR:26.3333 R:26.0000 rate:0.0520 aloss:1.3968 dloss:1.5741 aloss2:1.3200 exploreP:0.9922
Episode:3 meanR:23.0000 R:13.0000 rate:0.0260 aloss:1.4113 dloss:1.5487 aloss2:1.3366 exploreP:0.9909
Episode:4 meanR:27.0000 R:43.0000 rate:0.0860 aloss:1.4011 dloss:1.5394 aloss2:1.3439 exploreP:0.9867
Episode:5 meanR:24.5000 R:12.0000 rate:0.0240 aloss:1.4185 dloss:1.5235 aloss2:1.3494 exploreP:0.9856
Episode:6 meanR:22.8571 R:13.0000 rate:0.0260 aloss:1.4042 dloss:1.5076 aloss2:1.3521 exploreP:0.9843
Episode:7 meanR:22.3750 R:19.0000 rate:0.0380 aloss:1.3941 dloss:1.4970 aloss2:1.3695 exploreP:0.9824
Episode:8 meanR:22.0000 R:19.0000 rate:0.0380 aloss:1.3997 dloss:1.5034 aloss2:1.3498 exploreP:0.9806
Episode:9 meanR:21.5000 R:17.0000 rate:0.0340 aloss:1.3693 dloss:1.4530 aloss2:1.4

Episode:80 meanR:22.0494 R:17.0000 rate:0.0340 aloss:1.4380 dloss:0.7233 aloss2:2.8110 exploreP:0.8381
Episode:81 meanR:21.9512 R:14.0000 rate:0.0280 aloss:1.4630 dloss:0.7035 aloss2:2.7624 exploreP:0.8369
Episode:82 meanR:21.8554 R:14.0000 rate:0.0280 aloss:1.4709 dloss:0.6877 aloss2:2.8730 exploreP:0.8358
Episode:83 meanR:21.9048 R:26.0000 rate:0.0520 aloss:1.4166 dloss:0.7036 aloss2:2.8230 exploreP:0.8336
Episode:84 meanR:21.9176 R:23.0000 rate:0.0460 aloss:1.4414 dloss:0.7004 aloss2:2.8826 exploreP:0.8317
Episode:85 meanR:21.8488 R:16.0000 rate:0.0320 aloss:1.4177 dloss:0.6896 aloss2:2.8991 exploreP:0.8304
Episode:86 meanR:21.8391 R:21.0000 rate:0.0420 aloss:1.4160 dloss:0.6935 aloss2:2.8843 exploreP:0.8287
Episode:87 meanR:21.8295 R:21.0000 rate:0.0420 aloss:1.4134 dloss:0.6813 aloss2:2.9676 exploreP:0.8270
Episode:88 meanR:21.8090 R:20.0000 rate:0.0400 aloss:1.4265 dloss:0.6871 aloss2:3.0373 exploreP:0.8253
Episode:89 meanR:22.0778 R:46.0000 rate:0.0920 aloss:1.4148 dloss:0.6621 

Episode:159 meanR:19.3300 R:23.0000 rate:0.0460 aloss:1.3958 dloss:0.4626 aloss2:5.1874 exploreP:0.7228
Episode:160 meanR:19.2900 R:13.0000 rate:0.0260 aloss:1.3822 dloss:0.4654 aloss2:5.0026 exploreP:0.7219
Episode:161 meanR:19.3000 R:16.0000 rate:0.0320 aloss:1.4250 dloss:0.4465 aloss2:5.3399 exploreP:0.7207
Episode:162 meanR:19.2700 R:12.0000 rate:0.0240 aloss:1.4103 dloss:0.4755 aloss2:5.3059 exploreP:0.7199
Episode:163 meanR:18.9800 R:11.0000 rate:0.0220 aloss:1.3836 dloss:0.5435 aloss2:5.2658 exploreP:0.7191
Episode:164 meanR:18.9200 R:12.0000 rate:0.0240 aloss:1.3903 dloss:0.4767 aloss2:5.1514 exploreP:0.7183
Episode:165 meanR:18.9900 R:20.0000 rate:0.0400 aloss:1.3819 dloss:0.4691 aloss2:4.9861 exploreP:0.7168
Episode:166 meanR:19.1600 R:31.0000 rate:0.0620 aloss:1.3953 dloss:0.4824 aloss2:5.1649 exploreP:0.7147
Episode:167 meanR:19.0400 R:30.0000 rate:0.0600 aloss:1.3856 dloss:0.4777 aloss2:5.1986 exploreP:0.7125
Episode:168 meanR:19.0600 R:16.0000 rate:0.0320 aloss:1.4232 dlo

Episode:238 meanR:19.5700 R:17.0000 rate:0.0340 aloss:1.4159 dloss:0.4935 aloss2:5.5063 exploreP:0.6192
Episode:239 meanR:19.3700 R:13.0000 rate:0.0260 aloss:1.4130 dloss:0.4871 aloss2:5.4322 exploreP:0.6184
Episode:240 meanR:19.3300 R:12.0000 rate:0.0240 aloss:1.4144 dloss:0.4590 aloss2:5.5297 exploreP:0.6177
Episode:241 meanR:19.3500 R:22.0000 rate:0.0440 aloss:1.4143 dloss:0.4907 aloss2:5.4055 exploreP:0.6164
Episode:242 meanR:19.3600 R:12.0000 rate:0.0240 aloss:1.4083 dloss:0.4748 aloss2:5.4356 exploreP:0.6157
Episode:243 meanR:19.2700 R:21.0000 rate:0.0420 aloss:1.4107 dloss:0.5134 aloss2:5.4652 exploreP:0.6144
Episode:244 meanR:19.2800 R:14.0000 rate:0.0280 aloss:1.4277 dloss:0.4744 aloss2:5.6779 exploreP:0.6135
Episode:245 meanR:19.3800 R:27.0000 rate:0.0540 aloss:1.4239 dloss:0.4959 aloss2:5.6462 exploreP:0.6119
Episode:246 meanR:19.2100 R:16.0000 rate:0.0320 aloss:1.4133 dloss:0.4958 aloss2:5.5100 exploreP:0.6109
Episode:247 meanR:19.0500 R:13.0000 rate:0.0260 aloss:1.4302 dlo

Episode:317 meanR:26.1200 R:33.0000 rate:0.0660 aloss:1.3839 dloss:0.4676 aloss2:5.5039 exploreP:0.5076
Episode:318 meanR:26.5000 R:51.0000 rate:0.1020 aloss:1.3874 dloss:0.4884 aloss2:5.5315 exploreP:0.5051
Episode:319 meanR:26.7000 R:32.0000 rate:0.0640 aloss:1.3826 dloss:0.4854 aloss2:5.5368 exploreP:0.5035
Episode:320 meanR:26.8900 R:31.0000 rate:0.0620 aloss:1.3859 dloss:0.4819 aloss2:5.4775 exploreP:0.5020
Episode:321 meanR:27.0200 R:38.0000 rate:0.0760 aloss:1.3789 dloss:0.4628 aloss2:5.4199 exploreP:0.5001
Episode:322 meanR:27.0200 R:33.0000 rate:0.0660 aloss:1.3895 dloss:0.4807 aloss2:5.5609 exploreP:0.4985
Episode:323 meanR:26.4200 R:29.0000 rate:0.0580 aloss:1.3863 dloss:0.4610 aloss2:5.5108 exploreP:0.4971
Episode:324 meanR:27.0000 R:83.0000 rate:0.1660 aloss:1.3895 dloss:0.4718 aloss2:5.5412 exploreP:0.4930
Episode:325 meanR:28.1700 R:138.0000 rate:0.2760 aloss:1.3852 dloss:0.4756 aloss2:5.5149 exploreP:0.4864
Episode:326 meanR:28.3600 R:80.0000 rate:0.1600 aloss:1.3914 dl

...

Episode:2109 meanR:97.9300 R:96.0000 rate:0.1920 aloss:1.0092 dloss:0.8617 aloss2:4.2645 exploreP:0.0100
Episode:2110 meanR:97.5800 R:100.0000 rate:0.2000 aloss:1.0056 dloss:0.8552 aloss2:4.2428 exploreP:0.0100
Episode:2111 meanR:97.7600 R:75.0000 rate:0.1500 aloss:1.0052 dloss:0.8721 aloss2:4.2611 exploreP:0.0100
Episode:2112 meanR:97.8700 R:97.0000 rate:0.1940 aloss:1.0120 dloss:0.8656 aloss2:4.1374 exploreP:0.0100
Episode:2113 meanR:97.8300 R:77.0000 rate:0.1540 aloss:1.0128 dloss:0.8628 aloss2:4.2311 exploreP:0.0100
Episode:2114 meanR:98.0200 R:94.0000 rate:0.1880 aloss:1.0112 dloss:0.8487 aloss2:4.2854 exploreP:0.0100
Episode:2115 meanR:97.9000 R:92.0000 rate:0.1840 aloss:1.0015 dloss:0.8423 aloss2:4.3277 exploreP:0.0100
Episode:2116 meanR:98.7000 R:222.0000 rate:0.4440 aloss:1.0106 dloss:0.8600 aloss2:4.2457 exploreP:0.0100
Episode:2117 meanR:98.0000 R:64.0000 rate:0.1280 aloss:1.0055 dloss:0.8646 aloss2:4.2683 exploreP:0.0100
Episode:2118 meanR:97.8600 R:92.0000 rate:0.1840 alos

Episode:2187 meanR:101.0800 R:69.0000 rate:0.1380 aloss:1.0133 dloss:0.8600 aloss2:4.4350 exploreP:0.0100
Episode:2188 meanR:102.0200 R:184.0000 rate:0.3680 aloss:1.0154 dloss:0.8560 aloss2:4.4474 exploreP:0.0100
Episode:2189 meanR:102.6800 R:133.0000 rate:0.2660 aloss:1.0051 dloss:0.8512 aloss2:4.5166 exploreP:0.0100
Episode:2190 meanR:103.6900 R:168.0000 rate:0.3360 aloss:1.0080 dloss:0.8451 aloss2:4.5186 exploreP:0.0100
Episode:2191 meanR:103.1400 R:83.0000 rate:0.1660 aloss:1.0099 dloss:0.8679 aloss2:4.4278 exploreP:0.0100
Episode:2192 meanR:103.1900 R:82.0000 rate:0.1640 aloss:1.0103 dloss:0.8348 aloss2:4.4936 exploreP:0.0100
Episode:2193 meanR:102.0300 R:85.0000 rate:0.1700 aloss:1.0078 dloss:0.8396 aloss2:4.5875 exploreP:0.0100
Episode:2194 meanR:101.9500 R:53.0000 rate:0.1060 aloss:1.0073 dloss:0.8571 aloss2:4.5444 exploreP:0.0100
Episode:2195 meanR:101.1200 R:100.0000 rate:0.2000 aloss:1.0086 dloss:0.8535 aloss2:4.5855 exploreP:0.0100
Episode:2196 meanR:102.0300 R:189.0000 rat

Episode:2265 meanR:89.6400 R:64.0000 rate:0.1280 aloss:1.0110 dloss:0.8574 aloss2:4.5045 exploreP:0.0100
Episode:2266 meanR:89.6300 R:49.0000 rate:0.0980 aloss:1.0069 dloss:0.8546 aloss2:4.4438 exploreP:0.0100
Episode:2267 meanR:88.5500 R:47.0000 rate:0.0940 aloss:1.0148 dloss:0.8234 aloss2:4.5563 exploreP:0.0100
Episode:2268 meanR:87.8900 R:90.0000 rate:0.1800 aloss:1.0209 dloss:0.8551 aloss2:4.5960 exploreP:0.0100
Episode:2269 meanR:87.9200 R:84.0000 rate:0.1680 aloss:1.0125 dloss:0.8357 aloss2:4.6930 exploreP:0.0100
Episode:2270 meanR:87.8200 R:45.0000 rate:0.0900 aloss:0.9900 dloss:0.8352 aloss2:4.8286 exploreP:0.0100
Episode:2271 meanR:87.6100 R:50.0000 rate:0.1000 aloss:1.0026 dloss:0.8470 aloss2:4.6496 exploreP:0.0100
Episode:2272 meanR:87.6000 R:56.0000 rate:0.1120 aloss:1.0174 dloss:0.8616 aloss2:4.5663 exploreP:0.0100
Episode:2273 meanR:87.4300 R:78.0000 rate:0.1560 aloss:1.0126 dloss:0.8720 aloss2:4.4338 exploreP:0.0100
Episode:2274 meanR:87.3300 R:140.0000 rate:0.2800 aloss

Episode:2343 meanR:82.6200 R:52.0000 rate:0.1040 aloss:1.0162 dloss:0.8575 aloss2:4.4474 exploreP:0.0100
Episode:2344 meanR:82.6900 R:62.0000 rate:0.1240 aloss:1.0077 dloss:0.8606 aloss2:4.5624 exploreP:0.0100
Episode:2345 meanR:82.6900 R:152.0000 rate:0.3040 aloss:1.0138 dloss:0.8599 aloss2:4.5260 exploreP:0.0100
Episode:2346 meanR:82.7000 R:58.0000 rate:0.1160 aloss:1.0117 dloss:0.8575 aloss2:4.5128 exploreP:0.0100
Episode:2347 meanR:82.6100 R:58.0000 rate:0.1160 aloss:1.0135 dloss:0.8451 aloss2:4.6114 exploreP:0.0100
Episode:2348 meanR:82.5400 R:59.0000 rate:0.1180 aloss:1.0148 dloss:0.8551 aloss2:4.5694 exploreP:0.0100
Episode:2349 meanR:82.7300 R:59.0000 rate:0.1180 aloss:1.0146 dloss:0.8735 aloss2:4.4570 exploreP:0.0100
Episode:2350 meanR:82.8600 R:60.0000 rate:0.1200 aloss:1.0086 dloss:0.8649 aloss2:4.5432 exploreP:0.0100
Episode:2351 meanR:81.8900 R:61.0000 rate:0.1220 aloss:1.0076 dloss:0.8332 aloss2:4.6049 exploreP:0.0100
Episode:2352 meanR:82.6700 R:130.0000 rate:0.2600 alos

Episode:2421 meanR:80.0700 R:47.0000 rate:0.0940 aloss:1.0165 dloss:0.8480 aloss2:4.4800 exploreP:0.0100
Episode:2422 meanR:81.3000 R:174.0000 rate:0.3480 aloss:1.0204 dloss:0.8523 aloss2:4.5594 exploreP:0.0100
Episode:2423 meanR:81.1900 R:92.0000 rate:0.1840 aloss:1.0187 dloss:0.8502 aloss2:4.5686 exploreP:0.0100
Episode:2424 meanR:81.9700 R:138.0000 rate:0.2760 aloss:1.0166 dloss:0.8486 aloss2:4.4761 exploreP:0.0100
Episode:2425 meanR:81.9800 R:53.0000 rate:0.1060 aloss:1.0210 dloss:0.8307 aloss2:4.5827 exploreP:0.0100
Episode:2426 meanR:82.0600 R:69.0000 rate:0.1380 aloss:1.0196 dloss:0.8634 aloss2:4.5139 exploreP:0.0100
Episode:2427 meanR:81.9100 R:62.0000 rate:0.1240 aloss:1.0174 dloss:0.8602 aloss2:4.5017 exploreP:0.0100
Episode:2428 meanR:82.6600 R:133.0000 rate:0.2660 aloss:1.0176 dloss:0.8359 aloss2:4.5903 exploreP:0.0100
Episode:2429 meanR:82.9200 R:86.0000 rate:0.1720 aloss:1.0189 dloss:0.8239 aloss2:4.6410 exploreP:0.0100
Episode:2430 meanR:81.7000 R:55.0000 rate:0.1100 alo

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
# eps, arr = np.array(dloss_list).T
# smoothed_arr = running_mean(arr, 10)
# plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
# plt.plot(eps, arr, color='grey', alpha=0.3)
# plt.xlabel('Episode')
# plt.ylabel('D losses')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 481.0
total_reward: 481.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.